# INCLUDES IMPROVED PREPROCESSING + FEATURE ENGINEERING + MODEL SELECTION + MODEL FINE TUNING

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../../data/raw/train_data.csv')

In [3]:
df.duplicated().sum()

236

In [4]:
df = df.drop_duplicates()

In [5]:
df.duplicated().sum()

0

In [6]:
missing_thresh = 0.70
missing_ratio = df.isnull().mean()
drop_missing = missing_ratio[missing_ratio > missing_thresh].index.tolist()

# Drop columns with only one unique value
drop_constant = [col for col in df.columns if df[col].nunique() == 1]
# Drop columns having string "not available in demo"
drop_demo_cols = [col for col in df.columns if df[col].astype(str).str.contains("not available in demo", case=False).all()]

drop_cols = list(set(drop_missing + drop_constant + drop_demo_cols))

cols_to_keep=['trafficSource.isTrueDirect', 'new_visits', 'totals.bounces', 'trafficSource.adwordsClickInfo.isVideoAd']
for col in cols_to_keep:
    drop_cols.remove(col)

df = df.drop(columns=drop_cols)

print("Columns Dropped: ", drop_cols)

Columns Dropped:  ['trafficSource.adContent', 'trafficSource.adwordsClickInfo.slot', 'socialEngagementType', 'device.language', 'device.browserVersion', 'device.mobileDeviceModel', 'device.mobileDeviceBranding', 'totals.visits', 'device.flashVersion', 'device.browserSize', 'device.screenColors', 'geoNetwork.networkLocation', 'device.operatingSystemVersion', 'locationZone', 'device.mobileInputSelector', 'browserMajor', 'device.screenResolution', 'screenSize', 'trafficSource.adwordsClickInfo.adNetworkType', 'device.mobileDeviceMarketingName', 'trafficSource.adwordsClickInfo.page']


In [7]:
df = df.dropna(subset=['date', 'sessionStart'])

In [8]:
# Replacing placeholder values with NaNs and NaN with 'missing'
df['geoNetwork.region'] = df['geoNetwork.region'].replace("not available in demo dataset", np.nan)
df["geoNetwork.region"] = df["geoNetwork.region"].replace({"(not set)":np.nan})

df["trafficSource.campaign"] = df["trafficSource.campaign"].replace({"(not set)":np.nan})
df["trafficSource.keyword"] = df["trafficSource.keyword"].replace({np.nan:"missing"})

In [9]:
df['totals.bounces'] = df['totals.bounces'].fillna(0)
df['new_visits'] = df['new_visits'].fillna(0)

df['pageViews'] = df['pageViews'].fillna(1)
df['totalHits'] = df['totalHits'].fillna(1)
df['sessionNumber'] = df['sessionNumber'].fillna(1)

df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].fillna(False)
df['trafficSource.adwordsClickInfo.isVideoAd'] = df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)

C:\Users\HP\AppData\Local\Temp\ipykernel_864\1927767471.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].fillna(False)
C:\Users\HP\AppData\Local\Temp\ipykernel_864\1927767471.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['trafficSource.adwordsClickInfo.isVideoAd'] = df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)


In [10]:
df['is_campaign_set'] = df['trafficSource.campaign'].notna().astype(int)
df['has_referral'] = df['trafficSource.referralPath'].notna().astype(int)

In [11]:
df['region_city'] = df['geoNetwork.region'].astype(str) + "_" + df['geoNetwork.city'].astype(str)
df['continent_subcontinent'] = df['geoNetwork.continent'].astype(str) + "_" + df['geoNetwork.subContinent'].astype(str)

In [12]:
# Engagement metric: ratio of pageViews to totalHits (+1 to avoid division by 0)
df['page_hit_ratio'] = df['pageViews'] / (df['totalHits'] + 1)
df['page_hit_ratio'] = df['page_hit_ratio'].fillna(0)

# Number of unique sessions per user
user_session_counts = df.groupby('userId')['sessionId'].nunique()
df['user_session_count'] = df['userId'].map(user_session_counts)

# Average purchase value per user
user_avg_purchase = df.groupby('userId')['purchaseValue'].mean()
df['avg_purchase_by_user'] = df['userId'].map(user_avg_purchase)

In [13]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['sessionStart'] = pd.to_datetime(df['sessionStart'], unit='s')

df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['hour'] = df['sessionStart'].dt.hour

# Now drop after extraction
df = df.drop(columns=['date', 'sessionStart'])

In [14]:
for col in ['sessionNumber', 'pageViews', 'totalHits']:
    Q1 = df[col].quantile(0.10)
    Q3 = df[col].quantile(0.90)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

In [15]:
browser_counts = df['browser'].value_counts()
valid_browsers = browser_counts[browser_counts >= 200].index

df['browser'] = df['browser'].apply(lambda x: x if x in valid_browsers else 'Other')

In [16]:
X = df.drop(columns='purchaseValue')
y = df['purchaseValue']

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train.shape

(92629, 38)

In [19]:
y_train.shape

(92629,)

In [20]:
X_test.shape

(23158, 38)

In [21]:
y_test.shape

(23158,)

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92629 entries, 1640 to 15799
Data columns (total 38 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   trafficSource.isTrueDirect                92629 non-null  bool   
 1   browser                                   92629 non-null  object 
 2   trafficSource.keyword                     92629 non-null  object 
 3   geoCluster                                92629 non-null  object 
 4   userId                                    92629 non-null  int64  
 5   trafficSource.campaign                    4621 non-null   object 
 6   geoNetwork.networkDomain                  92629 non-null  object 
 7   gclIdPresent                              92629 non-null  int64  
 8   sessionNumber                             92629 non-null  float64
 9   geoNetwork.region                         42370 non-null  object 
 10  trafficSource                       

In [23]:
num_cols = X_train.select_dtypes(include=['float64','int64','bool','int32']).columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

print("Numerical Columns: ", num_cols)
print("\nCategorical Columns: ", cat_cols)

Numerical Columns:  ['trafficSource.isTrueDirect', 'userId', 'gclIdPresent', 'sessionNumber', 'sessionId', 'trafficSource.adwordsClickInfo.isVideoAd', 'pageViews', 'totals.bounces', 'totalHits', 'device.isMobile', 'new_visits', 'is_campaign_set', 'has_referral', 'page_hit_ratio', 'user_session_count', 'avg_purchase_by_user', 'day_of_week', 'month', 'hour']

Categorical Columns:  ['browser', 'trafficSource.keyword', 'geoCluster', 'trafficSource.campaign', 'geoNetwork.networkDomain', 'geoNetwork.region', 'trafficSource', 'os', 'geoNetwork.subContinent', 'trafficSource.medium', 'locationCountry', 'geoNetwork.city', 'geoNetwork.metro', 'trafficSource.referralPath', 'deviceType', 'userChannel', 'geoNetwork.continent', 'region_city', 'continent_subcontinent']


In [24]:
# Choosing numerical columns that need scaling
num_cols = ['pageViews', 'totalHits', 'sessionNumber','user_session_count']

In [25]:
# Separating categorical columns with greater than 10 unique values
low_card_cat_cols = list()
high_card_cat_cols = list()

for col in cat_cols:
    if (X_train[col].nunique() < 10):
        low_card_cat_cols.append(col)
    else:
        high_card_cat_cols.append(col)

In [26]:
print("Numerical columns to scale: ", num_cols)
print()
print("Low cradinality columns to One-Hot encode: ", low_card_cat_cols)
print()
print("High cradinality columns to Target encode: ", high_card_cat_cols)

Numerical columns to scale:  ['pageViews', 'totalHits', 'sessionNumber', 'user_session_count']

Low cradinality columns to One-Hot encode:  ['geoCluster', 'geoNetwork.networkDomain', 'trafficSource.medium', 'deviceType', 'userChannel', 'geoNetwork.continent']

High cradinality columns to Target encode:  ['browser', 'trafficSource.keyword', 'trafficSource.campaign', 'geoNetwork.region', 'trafficSource', 'os', 'geoNetwork.subContinent', 'locationCountry', 'geoNetwork.city', 'geoNetwork.metro', 'trafficSource.referralPath', 'region_city', 'continent_subcontinent']


In [34]:
!pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [27]:
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Imputing missing values of numerical columns with Median and Scaling 
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

# Imputing missing values with Mode and One-Hot Encoding for low cardinality columns
low_card_cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('label_encoder', OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Inputing missing values with Mode and Target Encoding for high cardinality columns
high_card_cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', ce.TargetEncoder())  
])

# Combining pipelines in Column Transformer and passing the remainder columns as it is
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_cols),
    ('low_card_cat', low_card_cat_pipeline, low_card_cat_cols),
    ('high_card_cat', high_card_cat_pipeline, high_card_cat_cols)
]
, remainder='passthrough')

In [36]:
!pip uninstall scikit-learn

^C


In [32]:
!pip install scikit-learn==1.3.2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [29]:
# Fit and transform the training set
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train)
# Transform the validation set
X_test_preprocessed = preprocessor.transform(X_test)

print("Preprocessing Done")
print()
print("Preprocessed X_train shape: ", X_train_preprocessed.shape)
print("Preprocessed X_test shape: ", X_test_preprocessed.shape)

Preprocessing Done

Preprocessed X_train shape:  (92629, 64)
Preprocessed X_test shape:  (23158, 64)


In [42]:
# !pip install catboost

You should consider upgrading via the 'c:\users\hp\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [44]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score

models = {
    "LGBM": LGBMRegressor(random_state=42),
    "XGB": XGBRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
    "HistGB": HistGradientBoostingRegressor(random_state=42),
    "RF": RandomForestRegressor(random_state=42)
}

results = []

for name, model in models.items():
    try:
        # Fit the model
        model.fit(X_train_preprocessed, y_train)
        print(f'{name}: Fitted Successfully on Training Data!')
        
        # Make predictions
        y_pred = model.predict(X_test_preprocessed)
        print(f'{name}: Predictions Made Successfully on Test Data!')
        
        # Calculate R2 score
        score = r2_score(y_test, y_pred)
        print(f'{name}: R2 Score Calculated Successfully! → {score:.4f}\n')
        
        # Store results
        results.append({
            'Model': name,
            'R2 Score': score
        })
    except Exception as e:
        print(f"Error with {name}: {str(e)}")
        results.append({
            'Model': name,
            'R2 Score': None,
            'Error': str(e)
        })

# Create a results dataframe
results_df = pd.DataFrame(results).sort_values(by='R2 Score', ascending=False)
print("\nFinal Results:")
print(results_df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019920 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2311
[LightGBM] [Info] Number of data points in the train set: 92629, number of used features: 62
[LightGBM] [Info] Start training from score 26057253.639702
LGBM: Fitted Successfully on Training Data!
LGBM: Predictions Made Successfully on Test Data!
LGBM: R2 Score Calculated Successfully! → 0.5666

XGB: Fitted Successfully on Training Data!
XGB: Predictions Made Successfully on Test Data!
XGB: R2 Score Calculated Successfully! → 0.5563

CatBoost: Fitted Successfully on Training Data!
CatBoost: Predictions Made Successfully on Test Data!
CatBoost: R2 Score Calculated Successfully! → 0.6821

HistGB: Fitted Successfully on Training Data!
HistGB: Predictions Made Successfully on Test Data!
HistGB: R2 Score Calculated Successfully! → 0.50

In [53]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 342.2 kB/s eta 0:00:05
   --------- ------------------------------ 0.5/2.1 MB 342.2 kB/s eta 0:00:05
   -------------- ------------------------- 0.8/2.1 MB 441.3 kB/s eta 0:00:04
   -------------- ------------------------- 0.8/2.1 MB 441.3 kB/s eta 0:00:04
   -------------- ------------------------- 0.8/2.1 MB 441.3 kB

In [48]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    model_name = trial.suggest_categorical("model", ["LGBM", "XGB", "CatBoost"])
    
    if model_name == "LGBM":
        params = {
            "n_estimators": trial.suggest_int("lgbm_n_estimators", 50, 500),
            "max_depth": trial.suggest_int("lgbm_max_depth", 3, 12),
            "learning_rate": trial.suggest_float("lgbm_lr", 1e-3, 0.3, log=True),
            "subsample": trial.suggest_float("lgbm_subsample", 0.6, 1.0),
        }
        model = LGBMRegressor(**params, random_state=42)
    
    elif model_name == "XGB":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 50, 500),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 12),
            "learning_rate": trial.suggest_float("xgb_lr", 1e-3, 0.3, log=True),
            "subsample": trial.suggest_float("xgb_subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample", 0.6, 1.0),
        }
        model = XGBRegressor(**params, random_state=42, tree_method="hist")
    
    elif model_name == "CatBoost":
        params = {
            "iterations": trial.suggest_int("catboost_iter", 50, 500),
            "depth": trial.suggest_int("catboost_depth", 3, 10),
            "learning_rate": trial.suggest_float("catboost_lr", 1e-3, 0.3, log=True),
        }
        model = CatBoostRegressor(**params, verbose=0, random_state=42)
    
    # Evaluate using cross-validation
    score = cross_val_score(model, X_train_preprocessed, y_train, cv=3, scoring="r2").mean()
    return score

In [49]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"Model: {trial.params['model']}")
print(f"R² Score: {trial.value:.4f}")
print("Best Params:", trial.params)

[I 2025-07-28 21:34:04,889] A new study created in memory with name: no-name-d84d057d-fa54-4607-8187-ed3305d4028b


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 61752, number of used features: 62
[LightGBM] [Info] Start training from score 25912868.049747
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2277
[LightGBM] [Info] Number of data points in the train set: 61753, number of used features: 62
[LightGBM] [Info] Start training from score 25408989.646884
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017271 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memor

[I 2025-07-28 21:34:15,019] Trial 0 finished with value: 0.5362325188127205 and parameters: {'model': 'LGBM', 'lgbm_n_estimators': 286, 'lgbm_max_depth': 11, 'lgbm_lr': 0.011421254259239707, 'lgbm_subsample': 0.8024226279692163}. Best is trial 0 with value: 0.5362325188127205.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030573 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 61752, number of used features: 62
[LightGBM] [Info] Start training from score 25912868.049747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[I 2025-07-28 21:34:24,718] Trial 1 finished with value: 0.5497393308693798 and parameters: {'model': 'LGBM', 'lgbm_n_estimators': 261, 'lgbm_max_depth': 8, 'lgbm_lr': 0.1192677448601505, 'lgbm_subsample': 0.842004637301245}. Best is trial 1 with value: 0.5497393308693798.
[I 2025-07-28 21:34:41,484] Trial 2 finished with value: 0.6347483865103197 and parameters: {'model': 'CatBoost', 'catboost_iter': 240, 'catboost_depth': 3, 'catboost_lr': 0.035199275490355446}. Best is trial 2 with value: 0.6347483865103197.
[I 2025-07-28 21:35:06,528] Trial 3 finished with value: 0.642666094490556 and parameters: {'model': 'CatBoost', 'catboost_iter': 458, 'catboost_depth': 3, 'catboost_lr': 0.1611865630589963}. Best is trial 3 with value: 0.642666094490556.
[I 2025-07-28 21:35:33,621] Trial 4 finished with value: 0.6823273013120129 and parameters: {'model': 'CatBoost', 'catboost_iter': 353, 'catboost_depth': 5, 'catboost_lr': 0.027612061881236633}. Best is trial 4 with value: 0.6823273013120129.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016983 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 61752, number of used features: 62
[LightGBM] [Info] Start training from score 25912868.049747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2025-07-28 21:59:12,370] Trial 40 finished with value: 0.05693906155438703 and parameters: {'model': 'LGBM', 'lgbm_n_estimators': 52, 'lgbm_max_depth': 4, 'lgbm_lr': 0.0011739911889834676, 'lgbm_subsample': 0.6025424534424533}. Best is trial 33 with value: 0.7162527565387317.
[I 2025-07-28 21:59:49,284] Trial 41 finished with value: 0.7058715165984571 and parameters: {'model': 'XGB', 'xgb_n_estimators': 336, 'xgb_max_depth': 11, 'xgb_lr': 0.018647931762692502, 'xgb_subsample': 0.9356950708488864, 'xgb_colsample': 0.6757754010047462}. Best is trial 33 with value: 0.7162527565387317.
[I 2025-07-28 22:00:15,710] Trial 42 finished with value: 0.6448302943643164 and parameters: {'model': 'XGB', 'xgb_n_estimators': 230, 'xgb_max_depth': 11, 'xgb_lr': 0.0078111996111867475, 'xgb_subsample': 0.9363555708475714, 'xgb_colsample': 0.661724273216223}. Best is trial 33 with value: 0.7162527565387317.
[I 2025-07-28 22:00:30,594] Trial 43 finished with value: 0.6922106976944007 and parameters: {'m

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 61752, number of used features: 62
[LightGBM] [Info] Start training from score 25912868.049747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2025-07-28 22:01:45,359] Trial 47 finished with value: 0.6193344396584779 and parameters: {'model': 'LGBM', 'lgbm_n_estimators': 483, 'lgbm_max_depth': 3, 'lgbm_lr': 0.26699445931408755, 'lgbm_subsample': 0.9711429459299696}. Best is trial 45 with value: 0.7176505614908782.
[I 2025-07-28 22:02:05,251] Trial 48 finished with value: 0.7017483094854301 and parameters: {'model': 'XGB', 'xgb_n_estimators': 342, 'xgb_max_depth': 8, 'xgb_lr': 0.09117342674936772, 'xgb_subsample': 0.7151616432747367, 'xgb_colsample': 0.8761637708429}. Best is trial 45 with value: 0.7176505614908782.
[I 2025-07-28 22:02:17,025] Trial 49 finished with value: 0.6993159603296167 and parameters: {'model': 'XGB', 'xgb_n_estimators': 257, 'xgb_max_depth': 6, 'xgb_lr': 0.04134590685742962, 'xgb_subsample': 0.7391052431276883, 'xgb_colsample': 0.7006615772180464}. Best is trial 45 with value: 0.7176505614908782.


Best trial:
Model: XGB
R² Score: 0.7177
Best Params: {'model': 'XGB', 'xgb_n_estimators': 343, 'xgb_max_depth': 7, 'xgb_lr': 0.05890776371030625, 'xgb_subsample': 0.7320325193461752, 'xgb_colsample': 0.6525959173893}


In [30]:
test = pd.read_csv('../../data/raw/test_data.csv')

In [31]:
cols_to_drop = ['trafficSource.adwordsClickInfo.adNetworkType', 'geoNetwork.networkLocation', 'browserMajor', 
                'device.mobileDeviceBranding', 'device.browserSize', 'socialEngagementType', 'trafficSource.adwordsClickInfo.page', 
                'locationZone', 'totals.visits', 'device.mobileDeviceModel', 'trafficSource.adwordsClickInfo.slot', 
                'device.mobileDeviceMarketingName', 'device.language', 'device.flashVersion', 'device.screenResolution', 'screenSize', 
                'device.operatingSystemVersion', 'device.browserVersion', 'trafficSource.adContent', 'device.screenColors', 
                'device.mobileInputSelector']

test = test.drop(columns=cols_to_drop)

In [32]:
# Replacing placeholder values with NaNs and NaN with 'missing'
test['geoNetwork.region'] = test['geoNetwork.region'].replace("not available in demo dataset", np.nan)
test["geoNetwork.region"] = test["geoNetwork.region"].replace({"(not set)":np.nan})

test["trafficSource.campaign"] = test["trafficSource.campaign"].replace({"(not set)":np.nan})
test["trafficSource.keyword"] = test["trafficSource.keyword"].replace({np.nan:"missing"})

In [33]:
test['totals.bounces'] = test['totals.bounces'].fillna(0)
test['new_visits'] = test['new_visits'].fillna(0)

test['pageViews'] = test['pageViews'].fillna(1)
test['totalHits'] = test['totalHits'].fillna(1)
test['sessionNumber'] = test['sessionNumber'].fillna(1)

test['trafficSource.isTrueDirect'] = test['trafficSource.isTrueDirect'].fillna(False)
test['trafficSource.adwordsClickInfo.isVideoAd'] = test['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)

C:\Users\HP\AppData\Local\Temp\ipykernel_864\2286199790.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['trafficSource.isTrueDirect'] = test['trafficSource.isTrueDirect'].fillna(False)
C:\Users\HP\AppData\Local\Temp\ipykernel_864\2286199790.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['trafficSource.adwordsClickInfo.isVideoAd'] = test['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)


In [34]:
test['is_campaign_set'] = test['trafficSource.campaign'].notna().astype(int)
test['has_referral'] = test['trafficSource.referralPath'].notna().astype(int)

In [35]:
test['region_city'] = test['geoNetwork.region'].astype(str) + "_" + test['geoNetwork.city'].astype(str)
test['continent_subcontinent'] = test['geoNetwork.continent'].astype(str) + "_" + test['geoNetwork.subContinent'].astype(str)

In [36]:
# Engagement metric: ratio of pageViews to totalHits (+1 to avoid division by 0)
test['page_hit_ratio'] = test['pageViews'] / (test['totalHits'] + 1)
test['page_hit_ratio'] = test['page_hit_ratio'].fillna(0)

# Number of unique sessions per user
test['user_session_count'] = test['userId'].map(user_session_counts)
# Fill unknown users with overall mean (just in case)
global_mean_sessions = user_session_counts.mean()
test['user_session_count'] = test['user_session_count'].fillna(global_mean_sessions)

# Average purchase value per user
test['avg_purchase_by_user'] = test['userId'].map(user_avg_purchase)
# Fill missing users (not seen in train) with overall average
overall_avg = user_avg_purchase.mean()
test['avg_purchase_by_user'] = test['avg_purchase_by_user'].fillna(overall_avg)

In [37]:
test['date'] = pd.to_datetime(test['date'], format='%Y%m%d')
test['sessionStart'] = pd.to_datetime(test['sessionStart'], unit='s')

test['day_of_week'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['hour'] = test['sessionStart'].dt.hour

# Now drop after extraction
test = test.drop(columns=['date', 'sessionStart'])

In [38]:
for col in ['sessionNumber', 'pageViews', 'totalHits']:
    Q1 = df[col].quantile(0.10)
    Q3 = df[col].quantile(0.90)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    test[col] = test[col].clip(lower=lower_bound, upper=upper_bound)

In [39]:
test['browser'] = test['browser'].apply(lambda x: x if x in valid_browsers else 'Other')

In [40]:
# Fit and transform the COMPLETE training set
X_preprocessed = preprocessor.fit_transform(X, y)
# Transform the testing set
test_preprocessed = preprocessor.transform(test)

print("Preprocessing Done")
print()
print("Preprocessed X shape: ", X_preprocessed.shape)
print("Preprocessed test shape: ", test_preprocessed.shape)

Preprocessing Done

Preprocessed X shape:  (115787, 64)
Preprocessed test shape:  (29006, 64)


In [41]:
# best_model = XGBRegressor(
#     n_estimators = 343,
#     max_depth = 7, 
#     lr = 0.05890776371030625, 
#     subsample = 0.7320325193461752, 
#     colsample = 0.6525959173893,
#     random_state = 42
# )

best_model = CatBoostRegressor(verbose=0, random_state=42)

best_model.fit(X_preprocessed, y)

real_y_pred = best_model.predict(test_preprocessed)
print(real_y_pred)

NameError: name 'CatBoostRegressor' is not defined

In [42]:
real_y_pred.shape

NameError: name 'real_y_pred' is not defined

In [73]:
# Prepare submission
submission_df = pd.DataFrame({
    'id': test.index,
    'purchaseValue': real_y_pred
})

submission_df.head()

,id,purchaseValue
0,0,2.688428e+07
1,1,1.095024e+07
2,2,1.294209e+07
3,3,-5.828922e+06
4,4,1.309141e+07


In [75]:
submission_df.to_csv('submission2.csv', index=False)

print("✅ Submission file created successfully!")
print(f"📁 Saved as: submission.csv")
print(f"📊 Sample predictions:\n{submission_df.head()}")

✅ Submission file created successfully!
📁 Saved as: submission.csv
📊 Sample predictions:
   id  purchaseValue
0   0   2.688428e+07
1   1   1.095024e+07
2   2   1.294209e+07
3   3  -5.828922e+06
4   4   1.309141e+07


In [41]:
from sklearn.preprocessing import StandardScaler

scale_cols = high_card_cat_cols

scaler = StandardScaler()

X_train_neural = X_train_preprocessed.copy()
X_test_neural = X_test_preprocessed.copy()

X_train_neural[:, :len(scale_cols)] = scaler.fit_transform(X_train_preprocessed[:, :len(scale_cols)])
X_test_neural[:, :len(scale_cols)] = scaler.transform(X_test_preprocessed[:, :len(scale_cols)])

In [42]:
X_train_neural.shape

(92629, 64)

In [43]:
X_test_neural.shape

(23158, 64)

In [44]:
X_train_neural = pd.DataFrame(X_train_neural)
X_test_neural = pd.DataFrame(X_test_neural)

In [45]:
X_train_neural = X_train_neural.astype('float32')
X_test_neural = X_test_neural.astype('float32')

In [46]:
X_train_neural.sample()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
40369,0.748303,0.882202,1.145422,-0.169932,-0.498316,-0.500307,1.998167,-0.502313,-0.498603,-0.708183,...,0.0,0.0,0.0,0.0,1.0,0.68,32070000.0,1.0,7.0,17.0


In [47]:
X_train_neural.shape

(92629, 64)

In [48]:
X_test_neural.shape

(23158, 64)

In [49]:
X_train_neural.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92629 entries, 0 to 92628
Data columns (total 64 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       92629 non-null  float32
 1   1       92629 non-null  float32
 2   2       92629 non-null  float32
 3   3       92629 non-null  float32
 4   4       92629 non-null  float32
 5   5       92629 non-null  float32
 6   6       92629 non-null  float32
 7   7       92629 non-null  float32
 8   8       92629 non-null  float32
 9   9       92629 non-null  float32
 10  10      92629 non-null  float32
 11  11      92629 non-null  float32
 12  12      92629 non-null  float32
 13  13      92629 non-null  float32
 14  14      92629 non-null  float32
 15  15      92629 non-null  float32
 16  16      92629 non-null  float32
 17  17      92629 non-null  float32
 18  18      92629 non-null  float32
 19  19      92629 non-null  float32
 20  20      92629 non-null  float32
 21  21      92629 non-null  float32
 22

In [50]:
# import tensorflow as tf
# from tensorflow.keras import backend as K

# def r2_metric(y_true, y_pred):
#     SS_res = K.sum(K.square(y_true - y_pred))
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
#     return 1 - SS_res / (SS_tot + K.epsilon())

In [51]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# # Model Architecture
# model = Sequential([
#     Dense(256, activation='relu', input_shape=(X_train_neural.shape[1],)),
#     BatchNormalization(),
#     Dropout(0.4),
#     Dense(128, activation='relu', kernel_regularizer='l2'),
#     BatchNormalization(),
#     Dropout(0.3),
#     Dense(64, activation='relu'),
#     Dense(1)  # Linear output for regression
# ])

# # Compile with R² tracking
# model.compile(
#     optimizer=Adam(learning_rate=0.001),
#     loss='mse',
#     metrics=[r2_metric]  # Now tracking R² during training
# )

# # Callbacks to maximize R²
# callbacks = [
#     EarlyStopping(monitor='val_r2_metric', patience=15, mode='max', restore_best_weights=True),
#     ReduceLROnPlateau(monitor='val_r2_metric', factor=0.5, patience=5, mode='max')
# ]

In [162]:
# history = model.fit(
#     X_train_neural, y_train,
#     validation_split=0.2,
#     epochs=200,
#     batch_size=512,
#     callbacks=callbacks,
#     verbose=1
# )

Epoch 1/200
145/145 [==============================] - 8s 27ms/step - loss: 43185170311806976.0000 - r2_metric: -0.0472 - val_loss: 24241877750382592.0000 - val_r2_metric: -0.0505 - lr: 0.0010
Epoch 2/200
145/145 [==============================] - 3s 23ms/step - loss: 43185002808082432.0000 - r2_metric: -0.0457 - val_loss: 24241733868978176.0000 - val_r2_metric: -0.0505 - lr: 0.0010
Epoch 3/200
145/145 [==============================] - 3s 22ms/step - loss: 43184637735862272.0000 - r2_metric: -0.0462 - val_loss: 24241467581005824.0000 - val_r2_metric: -0.0505 - lr: 0.0010
Epoch 4/200
145/145 [==============================] - 3s 22ms/step - loss: 43184066505211904.0000 - r2_metric: -0.0479 - val_loss: 24240668717088768.0000 - val_r2_metric: -0.0505 - lr: 0.0010
Epoch 5/200
145/145 [==============================] - 3s 22ms/step - loss: 43183108727504896.0000 - r2_metric: -0.0471 - val_loss: 24239556320559104.0000 - val_r2_metric: -0.0504 - lr: 0.0010
Epoch 6/200
145/145 [==============

In [1]:
# import tensorflow as tf
# import multiprocessing

# # Check GPU availability
# gpus = tf.config.list_physical_devices('GPU')
# print(f"Available GPUs: {gpus}")

# # Check CPU cores (for parallel trials)
# num_cores = multiprocessing.cpu_count()
# print(f"Available CPU cores: {num_cores}")

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Available CPU cores: 8


In [52]:
# import optuna
# from tensorflow.keras.regularizers import l1_l2
# from tensorflow.keras.optimizers import Nadam

# def objective(trial):
#     # Hyperparameter search space
#     params = {
#         'num_layers': trial.suggest_int('num_layers', 1, 4),
#         'units': trial.suggest_categorical('units', [64, 128, 256, 512]),
#         'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1),
#         'learning_rate': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
#         'l1_reg': trial.suggest_float('l1_reg', 1e-6, 1e-2, log=True),
#         'l2_reg': trial.suggest_float('l2_reg', 1e-6, 1e-2, log=True),
#         'batch_size': trial.suggest_categorical('batch_size', [128, 256, 512]),
#     }
    
#     # Build model
#     model = Sequential()
#     model.add(Dense(params['units'], activation='relu', 
#                    kernel_regularizer=l1_l2(params['l1_reg'], params['l2_reg']),
#                    input_shape=(X_train_neural.shape[1],)))
#     model.add(BatchNormalization())
#     model.add(Dropout(params['dropout_rate']))
    
#     # Add variable hidden layers
#     for _ in range(params['num_layers'] - 1):
#         model.add(Dense(params['units']//2, activation='relu',
#                       kernel_regularizer=l1_l2(params['l1_reg'], params['l2_reg'])))
#         model.add(BatchNormalization())
#         model.add(Dropout(params['dropout_rate']))
    
#     model.add(Dense(1))
    
#     # Compile
#     model.compile(
#         optimizer=Nadam(learning_rate=params['learning_rate']),
#         loss='mse',
#         metrics=[r2_metric]
#     )
    
#     # Train with early stopping
#     history = model.fit(
#         X_train_neural, y_train,
#         validation_split=0.2,
#         epochs=200,
#         batch_size=params['batch_size'],
#         callbacks=[
#             EarlyStopping(monitor='val_r2_metric', patience=15, mode='max', 
#                          restore_best_weights=True)
#         ],
#         verbose=0
#     )
    
#     # Return best validation R²
#     return max(history.history['val_r2_metric'])

In [50]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

In [51]:
import optuna
from sklearn.metrics import r2_score
import tensorflow as tf
from keras import Input
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1_l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [52]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU detected and configured:", gpus[0])
    except RuntimeError as e:
        print("❌ Error configuring GPU:", e)

✅ GPU detected and configured: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [53]:
# 🎯 Optuna Objective Function
def objective(trial):
    # 📌 Hyperparameter suggestions
    params = {
        'num_layers': trial.suggest_int('num_layers', 1, 4),
        'units': trial.suggest_categorical('units', [64, 128, 256, 512]),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1),
        'learning_rate': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'l1_reg': trial.suggest_float('l1_reg', 1e-6, 1e-2, log=True),
        'l2_reg': trial.suggest_float('l2_reg', 1e-6, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [128, 256, 512]),
    }

    # 🧠 Build ANN model
    model = Sequential()
    model.add(Input(shape=(X_train_neural.shape[1],)))
    model.add(Dense(params['units'],
                    activation='relu',
                    kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])))
    model.add(Dropout(params['dropout_rate']))

    for _ in range(params['num_layers'] - 1):
        model.add(Dense(params['units'], activation='relu',
                        kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])))
        model.add(Dropout(params['dropout_rate']))

    model.add(Dense(1))  # Output layer for regression

    # Compile with chosen learning rate
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']),
                  loss='mse')

    # 📉 EarlyStopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 🏋️‍♂️ Train model
    history = model.fit(
        X_train_neural, y_train,
        validation_data=(X_test_neural, y_test),
        batch_size=params['batch_size'],
        epochs=100,
        verbose=0,
        callbacks=[early_stop]
    )

    # 📈 Predict and evaluate
    y_pred = model.predict(X_test_neural, verbose=0)
    score = r2_score(y_test, y_pred)
    return score

In [54]:
# 🔍 Create and run Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, timeout=None)

# 🏆 Best trial
print("\n✅ Best R² Score:", study.best_value)
print("🔧 Best Hyperparameters:")
for k, v in study.best_params.items():
    print(f"   {k}: {v}")

[I 2025-07-30 07:40:55,873] A new study created in memory with name: no-name-620ef4c4-cc62-4ef2-8b7d-8dd8b861894f
[W 2025-07-30 07:40:56,770] Trial 0 failed with parameters: {'num_layers': 2, 'units': 512, 'dropout_rate': 0.5, 'lr': 0.0001588085239001558, 'l1_reg': 1.0162912469046449e-06, 'l2_reg': 0.0003270852262018246, 'batch_size': 128} because of the following error: UnknownError().
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\envs\tf_gpu\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_864\1227023991.py", line 17, in objective
    model.add(Dense(params['units'],
  File "C:\Users\HP\anaconda3\envs\tf_gpu\lib\site-packages\keras\src\models\sequential.py", line 122, in add
    self._maybe_rebuild()
  File "C:\Users\HP\anaconda3\envs\tf_gpu\lib\site-packages\keras\src\models\sequential.py", line 149, in _maybe_rebuild
    self.build(input_shape)
  File "C:\Users

UnknownError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} JIT compilation failed. [Op:FloorMod]

In [ ]:
# ✅ Best R² Score: 0.46689432241232864
# 🔧 Best Hyperparameters:
#    n_layers: 1
#    hidden_units: 512
#    dropout: 0.3616602393159346
#    learning_rate: 0.0022272108036892287
#    batch_size: 32
#    optimizer: rmsprop
#    patience: 15